In [73]:
import numpy as np
import os
import torch
import torch.nn as nn
import time
import pandas as pd
from scipy.stats import pearsonr

In [74]:
from model.util import Normalizer
from model.database_util import get_hist_file, get_job_table_sample, collator
from model.model import QueryFormer
from model.database_util import Encoding
from model.dataset import PlanTreeDataset
from model.trainer import eval_workload, train

In [75]:
data_path = './data/imdb/'

In [76]:
class Args:
    # bs = 1024
    # SQ: smaller batch size
    bs = 128
    lr = 0.001
    # epochs = 200
    epochs = 45
    clip_size = 50
    embed_size = 64
    pred_hid = 128
    ffn_dim = 128
    head_size = 12
    n_layers = 8
    dropout = 0.1
    sch_decay = 0.6
    # device = 'cuda:0'
    device = 'cpu'
    newpath = './results/full/cost/'
    to_predict = 'cost'
args = Args()

import os
if not os.path.exists(args.newpath):
    os.makedirs(args.newpath)

In [77]:
hist_file = get_hist_file(data_path + 'histogram_string.csv')
cost_norm = Normalizer(-3.61192, 12.290855)
card_norm = Normalizer(1,100)

/Users/shaikhq/coding/QueryFormer/model/database_util.py:76: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  hist_file['freq'][i] = freq_np
/Users/shaikhq/coding/QueryFormer/model/database_util.py:89: FutureWarning: ChainedAssignmentError: beh

In [78]:
encoding_ckpt = torch.load('checkpoints/encoding.pt')
type(encoding_ckpt)

dict

In [79]:
encoding_ckpt.keys()

dict_keys(['encoding'])

In [80]:
encoding = encoding_ckpt['encoding']
type(encoding)

model.database_util.Encoding

## Exploring Encoding object

In [81]:
print(encoding.column_min_max_vals)
# column_min_max_vals is a dictionary. It has the min and max value for each numeric column in the dataset. 

{'t.id': [1.0, 2528312.0], 't.kind_id': [1.0, 7.0], 't.production_year': [1880.0, 2019.0], 'mc.id': [1.0, 2609129.0], 'mc.company_id': [1.0, 234997.0], 'mc.movie_id': [2.0, 2525745.0], 'mc.company_type_id': [1.0, 2.0], 'ci.id': [1.0, 36244344.0], 'ci.movie_id': [1.0, 2525975.0], 'ci.person_id': [1.0, 4061926.0], 'ci.role_id': [1.0, 11.0], 'mi.id': [1.0, 14835720.0], 'mi.movie_id': [1.0, 2526430.0], 'mi.info_type_id': [1.0, 110.0], 'mi_idx.id': [1.0, 1380035.0], 'mi_idx.movie_id': [2.0, 2525793.0], 'mi_idx.info_type_id': [99.0, 113.0], 'mk.id': [1.0, 4523930.0], 'mk.movie_id': [2.0, 2525971.0], 'mk.keyword_id': [1.0, 134170.0]}


In [82]:
print(encoding.col2idx)
# the label encoding of each unique column in the dataset

{'t.id': 0, 't.kind_id': 1, 't.production_year': 2, 'mc.id': 3, 'mc.company_id': 4, 'mc.movie_id': 5, 'mc.company_type_id': 6, 'ci.id': 7, 'ci.movie_id': 8, 'ci.person_id': 9, 'ci.role_id': 10, 'mi.id': 11, 'mi.movie_id': 12, 'mi.info_type_id': 13, 'mi_idx.id': 14, 'mi_idx.movie_id': 15, 'mi_idx.info_type_id': 16, 'mk.id': 17, 'mk.movie_id': 18, 'mk.keyword_id': 19, 'NA': 20}


In [83]:
print(encoding.op2idx)

{'>': 0, '=': 1, '<': 2, 'NA': 3}


In [84]:
print(encoding.idx2col)

{0: 't.id', 1: 't.kind_id', 2: 't.production_year', 3: 'mc.id', 4: 'mc.company_id', 5: 'mc.movie_id', 6: 'mc.company_type_id', 7: 'ci.id', 8: 'ci.movie_id', 9: 'ci.person_id', 10: 'ci.role_id', 11: 'mi.id', 12: 'mi.movie_id', 13: 'mi.info_type_id', 14: 'mi_idx.id', 15: 'mi_idx.movie_id', 16: 'mi_idx.info_type_id', 17: 'mk.id', 18: 'mk.movie_id', 19: 'mk.keyword_id', 20: 'NA'}


In [85]:
print(encoding.type2idx)

{'Gather': 0, 'Hash Join': 1, 'Seq Scan': 2, 'Hash': 3, 'Bitmap Heap Scan': 4, 'Bitmap Index Scan': 5, 'Nested Loop': 6, 'Index Scan': 7, 'Merge Join': 8, 'Gather Merge': 9, 'Materialize': 10, 'BitmapAnd': 11, 'Sort': 12}


In [86]:
checkpoint = torch.load('checkpoints/cost_model.pt', map_location='cpu')

In [87]:
from model.util import seed_everything
seed_everything()

In [88]:
model = QueryFormer(emb_size = args.embed_size ,ffn_dim = args.ffn_dim, head_size = args.head_size, \
                 dropout = args.dropout, n_layers = args.n_layers, \
                 use_sample = True, use_hist = True, \
                 pred_hid = args.pred_hid
                )

In [89]:
_ = model.to(args.device)

In [90]:
to_predict = 'cost'

In [91]:
imdb_path = './data/imdb/'
dfs = []  # list to hold DataFrames
# SQ: added
for i in range(2):
#for i in range(18):
    file = imdb_path + 'plan_and_cost/train_plan_part{}.csv'.format(i)
    df = pd.read_csv(file)
    dfs.append(df)

full_train_df = pd.concat(dfs)

val_dfs = []  # list to hold DataFrames
for i in range(18,20):
    file = imdb_path + 'plan_and_cost/train_plan_part{}.csv'.format(i)
    df = pd.read_csv(file)
    val_dfs.append(df)

val_df = pd.concat(val_dfs)

In [92]:
table_sample = get_job_table_sample(imdb_path+'train')

Loaded queries with len  100000
Loaded bitmaps


## Exploring Histogram input (hist_file)

In [93]:
type(hist_file)

pandas.core.frame.DataFrame

In [94]:
hist_file.head()

,table,column,freq,bins,table_column
0,title,production_year,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1880, 1913, 1923, 1942, 1955, 1960, 1964, 196...",t.production_year
1,title,kind_id,"[0.0, 0.26216118191156074, 0.03593387047716835...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, ...",t.kind_id
2,movie_companies,company_id,"[0.0, 0.0004959511376981121, 0.000558807386989...","[1, 6, 19, 27, 68, 133, 160, 189, 292, 402, 47...",mc.company_id
3,movie_companies,company_type_id,"[0.0, 0.4883796425472418, 0.5116203574527581]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",mc.company_type_id
4,cast_info,role_id,"[0.0, 0.3495907485479872, 0.20560375449487386,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",ci.role_id


In [95]:
hist_file.shape

(9, 5)

In [96]:
hist_file.head(10)

,table,column,freq,bins,table_column
0,title,production_year,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1880, 1913, 1923, 1942, 1955, 1960, 1964, 196...",t.production_year
1,title,kind_id,"[0.0, 0.26216118191156074, 0.03593387047716835...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, ...",t.kind_id
2,movie_companies,company_id,"[0.0, 0.0004959511376981121, 0.000558807386989...","[1, 6, 19, 27, 68, 133, 160, 189, 292, 402, 47...",mc.company_id
3,movie_companies,company_type_id,"[0.0, 0.4883796425472418, 0.5116203574527581]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",mc.company_type_id
4,cast_info,role_id,"[0.0, 0.3495907485479872, 0.20560375449487386,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",ci.role_id
5,movie_keyword,keyword_id,"[0.0, 0.0031748950967179193, 1.989421142551088...","[1, 77, 132, 230, 331, 347, 384, 495, 643, 784...",mk.keyword_id
6,cast_info,person_id,"[0.0, 0.0, 5.518102507748589e-08, 2.7590512538...","[2, 77446, 145798, 212750, 281691, 347240, 419...",ci.person_id
7,movie_info_idx,info_type_id,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 9...",mi_idx.info_type_id
8,movie_info,info_type_id,"[0.0, 0.05406815807174563, 0.08688004942665738...","[1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, ...",mi.info_type_id


In [97]:
hist_file.dtypes

table           object
column          object
freq            object
bins            object
table_column    object
dtype: object

In [98]:
for column in ['table', 'column', 'freq', 'bins', 'table_column']:
    print(f"Column: {column}")
    print(hist_file[column].apply(type).value_counts())
    sample_value = hist_file[column].iloc[0]
    print(f"Sample Value: {sample_value}")
    if isinstance(sample_value, (list, tuple, set, dict, pd.Series, np.ndarray)):
        print(f"Length of Sample Value: {len(sample_value)}")
    print("\n")

Column: table
table
<class 'str'>    9
Name: count, dtype: int64
Sample Value: title


Column: column
column
<class 'str'>    9
Name: count, dtype: int64
Sample Value: production_year


Column: freq
freq
<class 'numpy.ndarray'>    9
Name: count, dtype: int64
Sample Value: [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.22139046e-06
 0.00000000e+00 8.14260304e-07]
Length of Sample Value: 2020


Column: bins
bins
<class 'list'>    9
Name: count, dtype: int64
Sample Value: [1880, 1913, 1923, 1942, 1955, 1960, 1964, 1968, 1971, 1975, 1978, 1982, 1985, 1987, 1990, 1992, 1994, 1995, 1996, 1998, 1999, 2000, 2001, 2001, 2002, 2003, 2004, 2004, 2005, 2005, 2006, 2006, 2007, 2007, 2007, 2008, 2008, 2009, 2009, 2009, 2010, 2010, 2010, 2011, 2011, 2011, 2012, 2012, 2012, 2013, 2019]
Length of Sample Value: 51


Column: table_column
table_column
<class 'str'>    9
Name: count, dtype: int64
Sample Value: t.production_year




# Step 1: Identifying the training dataset and its component

In [99]:
train_ds = PlanTreeDataset(full_train_df, None, encoding, hist_file, card_norm, cost_norm, to_predict, table_sample)

# Step 2: Exploring full_train_df

In [100]:
type(full_train_df)

pandas.core.frame.DataFrame

In [101]:
full_train_df.shape

(10000, 2)

In [102]:
full_train_df.head(2)

,id,json
0,0,"{""Plan"": {""Node Type"": ""Gather"", ""Parallel Awa..."
1,1,"{""Plan"": {""Node Type"": ""Seq Scan"", ""Parallel A..."


In [103]:
full_train_df.dtypes

id       int64
json    object
dtype: object

In [104]:
# printing a sample json string in full
json_sample = full_train_df['json'].iloc[0]
print(json_sample)

{"Plan": {"Node Type": "Gather", "Parallel Aware": false, "Startup Cost": 23540.58, "Total Cost": 154548.95, "Plan Rows": 567655, "Plan Width": 119, "Actual Startup Time": 386.847, "Actual Total Time": 646.972, "Actual Rows": 283812, "Actual Loops": 1, "Workers Planned": 2, "Workers Launched": 2, "Single Copy": false, "Plans": [{"Node Type": "Hash Join", "Parent Relationship": "Outer", "Parallel Aware": true, "Join Type": "Inner", "Startup Cost": 22540.58, "Total Cost": 96783.45, "Plan Rows": 236523, "Plan Width": 119, "Actual Startup Time": 369.985, "Actual Total Time": 518.487, "Actual Rows": 94604, "Actual Loops": 3, "Inner Unique": false, "Hash Cond": "(t.id = mi_idx.movie_id)", "Workers": [], "Plans": [{"Node Type": "Seq Scan", "Parent Relationship": "Outer", "Parallel Aware": true, "Relation Name": "title", "Alias": "t", "Startup Cost": 0.0, "Total Cost": 49166.46, "Plan Rows": 649574, "Plan Width": 94, "Actual Startup Time": 0.366, "Actual Total Time": 147.047, "Actual Rows": 51

In [105]:
# pretty print the json object
import json
# the following code parses the json string into a dictionary
json_parsed = json.loads(json_sample)
json_pretty = json.dumps(json_parsed, indent=4)
print(json_pretty)

with open('output.json', 'w') as f:
    f.write(json_pretty)

{
    "Plan": {
        "Node Type": "Gather",
        "Parallel Aware": false,
        "Startup Cost": 23540.58,
        "Total Cost": 154548.95,
        "Plan Rows": 567655,
        "Plan Width": 119,
        "Actual Startup Time": 386.847,
        "Actual Total Time": 646.972,
        "Actual Rows": 283812,
        "Actual Loops": 1,
        "Workers Planned": 2,
        "Workers Launched": 2,
        "Single Copy": false,
        "Plans": [
            {
                "Node Type": "Hash Join",
                "Parent Relationship": "Outer",
                "Parallel Aware": true,
                "Join Type": "Inner",
                "Startup Cost": 22540.58,
                "Total Cost": 96783.45,
                "Plan Rows": 236523,
                "Plan Width": 119,
                "Actual Startup Time": 369.985,
                "Actual Total Time": 518.487,
                "Actual Rows": 94604,
                "Actual Loops": 3,
                "Inner Unique": false,
         

In [106]:
type(encoding)

model.database_util.Encoding

In [107]:
val_ds = PlanTreeDataset(val_df, None, encoding, hist_file, card_norm, cost_norm, to_predict, table_sample)

In [108]:
crit = nn.MSELoss()
model, best_path = train(model, train_ds, val_ds, crit, cost_norm, args)

Epoch: 0  Avg Loss: 0.00015329097853973507, Time: 41.42007398605347
Median: 2.928444377690787
Mean: 449.13245681912656
Epoch: 20  Avg Loss: 9.051527210976929e-06, Time: 890.048810005188
Median: 1.190994000093597
Mean: 1.6319231603649687
Epoch: 40  Avg Loss: 7.69963618076872e-06, Time: 1726.8186321258545
Median: 1.1766559018761633
Mean: 1.5472035230118144


In [109]:
methods = {
    'get_sample' : get_job_table_sample,
    'encoding': encoding,
    'cost_norm': cost_norm,
    'hist_file': hist_file,
    'model': model,
    'device': args.device,
    'bs': 512,
}

In [110]:
_ = eval_workload('job-light', methods)

Loaded queries with len  70
Loaded bitmaps
Median: 1.6471430303309913
Mean: 19.293510404012288
Corr:  0.8708629547656623


In [111]:
_ = eval_workload('synthetic', methods)

Loaded queries with len  5000
Loaded bitmaps
Median: 1.1351381926301887
Mean: 1.6124878862677317
Corr:  0.9828999983362362
